# 2024 Bike Availability Prediction

In [1]:
# Importar librerías que vamos a estar usando en general
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from tqdm import tqdm, trange


## Load data

In [2]:
months = ['Gener', 'Febrer', 'Marc', 'Abril', 'Maig', 'Juny', 'Juliol', 'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre']
indeces = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
years = ['2023'] #['2021', '2020'] # '2021', '2022',

## OJO ##
# EN 2021 Y 2022 EL DOCUMENTO SE LLAMA MARÇ EN VEZ DE MARC, HAY QUE CAMBIARLO A MARC PARA QUE FUNCIONE
pth = "/Users/albam/Desktop/UNIVERSITAT/POSTGRAU/CAPSTONE_PROJECT/raw_data"
data = pd.DataFrame()
for year in years:
    for index, month in zip(indeces, months):
        month_year = pd.read_csv((pth + "/{}_{}_{}_BicingNou_ESTACIONS.csv").format(year, index, month))
        data = pd.concat([data, month_year])

# Visualizar la tabla de datos completa
data.head()

/var/folders/z9/sss87zk9299_8096st9s8gnh0000gn/T/ipykernel_7858/3161911499.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  month_year = pd.read_csv((pth + "/{}_{}_{}_BicingNou_ESTACIONS.csv").format(year, index, month))
/var/folders/z9/sss87zk9299_8096st9s8gnh0000gn/T/ipykernel_7858/3161911499.py:11: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  month_year = pd.read_csv((pth + "/{}_{}_{}_BicingNou_ESTACIONS.csv").format(year, index, month))


,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,traffic,last_updated,ttl,V1
0,1.0,38.0,35.0,3.0,8.0,1.672528e+09,True,IN_SERVICE,1.0,1.0,1.0,NaN,1.672528e+09,19.0,NaN
1,2.0,6.0,5.0,1.0,23.0,1.672527e+09,True,IN_SERVICE,1.0,1.0,1.0,NaN,1.672528e+09,19.0,NaN
2,3.0,20.0,20.0,0.0,7.0,1.672527e+09,True,IN_SERVICE,1.0,1.0,1.0,NaN,1.672528e+09,19.0,NaN
3,4.0,19.0,19.0,0.0,2.0,1.672528e+09,True,IN_SERVICE,1.0,1.0,1.0,NaN,1.672528e+09,19.0,NaN
4,5.0,4.0,3.0,1.0,35.0,1.672527e+09,True,IN_SERVICE,1.0,1.0,1.0,NaN,1.672528e+09,19.0,NaN


In [3]:
data['last_updated'] = pd.to_datetime(data['last_updated'], unit='s')
data['last_reported'] = pd.to_datetime(data['last_reported'], unit='s')

/Users/albam/Applications/anaconda3/lib/python3.11/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/Users/albam/Applications/anaconda3/lib/python3.11/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


In [4]:
data.columns

Index(['station_id', 'num_bikes_available',
       'num_bikes_available_types.mechanical',
       'num_bikes_available_types.ebike', 'num_docks_available',
       'last_reported', 'is_charging_station', 'status', 'is_installed',
       'is_renting', 'is_returning', 'traffic', 'last_updated', 'ttl', 'V1'],
      dtype='object')

In [5]:
for column in data.columns:
    clases = data[column].unique()
    print(column, clases)

station_id [  1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.
  30.  31.  32.  33.  34.  35.  36.  37.  39.  40.  41.  42.  43.  44.
  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.
  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.  71.  72.  73.
  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.  84.  85.  86.  87.
  88.  89.  90.  91.  92.  94.  95.  97.  98.  99. 100. 101. 102. 103.
 104. 105. 106. 107. 108. 109. 110. 111. 112. 113. 114. 115. 116. 117.
 118. 119. 120. 121. 122. 123. 124. 125. 126. 127. 128. 129. 130. 131.
 132. 133. 134. 135. 136. 137. 138. 139. 140. 141. 142. 143. 144. 145.
 146. 147. 148. 149. 150. 151. 152. 153. 154. 155. 156. 157. 158. 159.
 161. 162. 163. 164. 165. 166. 167. 168. 170. 171. 173. 174. 175. 176.
 177. 178. 179. 180. 182. 183. 184. 185. 186. 187. 188. 189. 190. 192.
 193. 194. 195. 196. 197. 198. 199. 200. 201. 202. 203. 204. 205. 

In [6]:
data['year'] = data['last_updated'].dt.year
data['month'] = data['last_updated'].dt.month
data['day'] = data['last_updated'].dt.day
data['hour'] = data['last_updated'].dt.hour

data.drop(columns=['last_updated'], inplace=True)

In [7]:
months_dict = {1: 'Gener', 2: 'Febrer', 3: 'Marc', 4: 'Abril', 5: 'Maig',
            6: 'Juny', 7: 'Juliol', 8: 'Agost', 9: 'Setembre', 10: 'Octubre', 11: 'Novembre', 12: 'Desembre'}

data['month_name'] = data['month'].map(months_dict)
data.head()

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,traffic,ttl,V1,year,month,day,hour,month_name
0,1.0,38.0,35.0,3.0,8.0,2022-12-31 22:59:21,True,IN_SERVICE,1.0,1.0,1.0,NaN,19.0,NaN,2022.0,12.0,31.0,22.0,Desembre
1,2.0,6.0,5.0,1.0,23.0,2022-12-31 22:56:31,True,IN_SERVICE,1.0,1.0,1.0,NaN,19.0,NaN,2022.0,12.0,31.0,22.0,Desembre
2,3.0,20.0,20.0,0.0,7.0,2022-12-31 22:57:20,True,IN_SERVICE,1.0,1.0,1.0,NaN,19.0,NaN,2022.0,12.0,31.0,22.0,Desembre
3,4.0,19.0,19.0,0.0,2.0,2022-12-31 22:58:36,True,IN_SERVICE,1.0,1.0,1.0,NaN,19.0,NaN,2022.0,12.0,31.0,22.0,Desembre
4,5.0,4.0,3.0,1.0,35.0,2022-12-31 22:55:42,True,IN_SERVICE,1.0,1.0,1.0,NaN,19.0,NaN,2022.0,12.0,31.0,22.0,Desembre


## Visualization

def vis_years_months(years, months, data, x, y, xlab, ylab):
    for year in years:
        for month in months:
            average_bikes = data[data['month_name'] == month].groupby(x)[y].mean()
            plt.plot(average_bikes, label=month)
        plt.ylabel(ylab)
        plt.xlabel(xlab)
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
        plt.title(str(year))
        plt.show()  

# Nombre de bicicletes disponibles (mitjana) en funció de l'hora per cada mes

#num_rows = (len(months) - 1) // 3 + 1
#fig, axes = plt.subplots(num_rows, 3, figsize=(18, 4 * num_rows))
#axes = axes.flatten()

vis_years_months(years, months, data, 'hour', 'num_bikes_available', 'Hora', "Mitjana de bicicletes disponibles")  

# Nombre de bicicletes disponibles (mitjana) en funció del dia per cada mes

vis_years_months(years, months, data, 'day', 'num_bikes_available', 'Dia del mes', "Mitjana de bicicletes disponibles")  

# Nombre de bicicletes MECÀNIQUES disponibles (mitjana) en funció de l'hora per cada mes

vis_years_months(years, months, data, 'hour', 'num_bikes_available_types.mechanical', 'Hora', "Mitjana de bicicletes mecàniques disponibles")  

# Nombre de bicicletes ELÈCTRIQUES disponibles (mitjana) en funció de l'hora per cada mes

vis_years_months(years, months, data, 'hour', 'num_bikes_available_types.ebike', 'Hora', "Mitjana de bicicletes elèctriques disponibles")  

## Data cleaning

In [8]:
def nan_count(data):
    # Comptar nans
    nan_count = data.isnull().sum()

    # Visualitzar files amb valors NaN
    print("Hi ha Nans:")
    print(nan_count)

nan_count(data) 

Hi ha Nans:
station_id                                  3765
num_bikes_available                         3765
num_bikes_available_types.mechanical        3765
num_bikes_available_types.ebike             3765
num_docks_available                         3765
last_reported                               3765
is_charging_station                         3765
status                                      3765
is_installed                                3765
is_renting                                  3765
is_returning                                3765
traffic                                 49193647
ttl                                         3765
V1                                      49193647
year                                        3765
month                                       3765
day                                         3765
hour                                        3765
month_name                                  3765
dtype: int64


Veiem que les columnes traffic i V1 estan buides, i que hi ha files buides on tots els valors son NaNs. Ho treiem.

In [9]:
data.drop(columns=['V1', 'traffic'], inplace=True) # Eliminar columnes amb valors NaN

In [10]:
data.dropna(inplace=True) # Eliminar files amb valors NaN 
data

,station_id,num_bikes_available,num_bikes_available_types.mechanical,num_bikes_available_types.ebike,num_docks_available,last_reported,is_charging_station,status,is_installed,is_renting,is_returning,ttl,year,month,day,hour,month_name
0,1.0,38.0,35.0,3.0,8.0,2022-12-31 22:59:21,True,IN_SERVICE,1.0,1.0,1.0,19.0,2022.0,12.0,31.0,22.0,Desembre
1,2.0,6.0,5.0,1.0,23.0,2022-12-31 22:56:31,True,IN_SERVICE,1.0,1.0,1.0,19.0,2022.0,12.0,31.0,22.0,Desembre
2,3.0,20.0,20.0,0.0,7.0,2022-12-31 22:57:20,True,IN_SERVICE,1.0,1.0,1.0,19.0,2022.0,12.0,31.0,22.0,Desembre
3,4.0,19.0,19.0,0.0,2.0,2022-12-31 22:58:36,True,IN_SERVICE,1.0,1.0,1.0,19.0,2022.0,12.0,31.0,22.0,Desembre
4,5.0,4.0,3.0,1.0,35.0,2022-12-31 22:55:42,True,IN_SERVICE,1.0,1.0,1.0,19.0,2022.0,12.0,31.0,22.0,Desembre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4525541,515.0,5.0,5.0,0.0,19.0,2023-12-31 23:12:57,True,IN_SERVICE,1.0,1.0,1.0,1.0,2023.0,12.0,31.0,23.0,Desembre
4525542,516.0,10.0,2.0,8.0,10.0,2023-12-31 23:11:18,True,IN_SERVICE,1.0,1.0,1.0,1.0,2023.0,12.0,31.0,23.0,Desembre
4525543,517.0,8.0,1.0,7.0,11.0,2023-12-31 23:14:58,True,IN_SERVICE,1.0,1.0,1.0,1.0,2023.0,12.0,31.0,23.0,Desembre
4525544,518.0,3.0,1.0,2.0,24.0,2023-12-31 23:14:05,True,IN_SERVICE,1.0,1.0,1.0,1.0,2023.0,12.0,31.0,23.0,Desembre


In [11]:
nan_count(data)

Hi ha Nans:
station_id                              0
num_bikes_available                     0
num_bikes_available_types.mechanical    0
num_bikes_available_types.ebike         0
num_docks_available                     0
last_reported                           0
is_charging_station                     0
status                                  0
is_installed                            0
is_renting                              0
is_returning                            0
ttl                                     0
year                                    0
month                                   0
day                                     0
hour                                    0
month_name                              0
dtype: int64


Ara que no tenim NaNs, comencem amb al format del df: treurem les columnes que a priori no farem servir i n'afegim de noves. Juntarem el df amb el de la informació de les estacions.

In [12]:
data['percentage_docks_available'] = data['num_docks_available'] / (data['num_bikes_available'] + data['num_docks_available'])

output_data = data[['station_id','last_reported', 'year','month', 'day', 'hour', 'percentage_docks_available']].copy()
output_data.reset_index(drop=False, inplace=True)
output_data.rename(columns={'index': 'index'}, inplace=True)

int_cols = ['station_id', 'year','month', 'day', 'hour']
for col in int_cols:
    output_data[col] = output_data[col].astype(int)
    
output_data

,index,station_id,last_reported,year,month,day,hour,percentage_docks_available
0,0,1,2022-12-31 22:59:21,2022,12,31,22,0.173913
1,1,2,2022-12-31 22:56:31,2022,12,31,22,0.793103
2,2,3,2022-12-31 22:57:20,2022,12,31,22,0.259259
3,3,4,2022-12-31 22:58:36,2022,12,31,22,0.095238
4,4,5,2022-12-31 22:55:42,2022,12,31,22,0.897436
...,...,...,...,...,...,...,...,...
49189877,4525541,515,2023-12-31 23:12:57,2023,12,31,23,0.791667
49189878,4525542,516,2023-12-31 23:11:18,2023,12,31,23,0.500000
49189879,4525543,517,2023-12-31 23:14:58,2023,12,31,23,0.578947
49189880,4525544,518,2023-12-31 23:14:05,2023,12,31,23,0.888889


In [13]:
info_stations = '/Users/albam/Desktop/UNIVERSITAT/POSTGRAU/CAPSTONE_PROJECT/Gorrapiedra-proyecto/data/info_bicing/Informacio_Estacions_Bicing.csv'

df_info_stations = pd.read_csv(info_stations)

df_info_stations

,station_id,name,physical_configuration,lat,lon,altitude,address,post_code,capacity,is_charging_station,nearby_distance,_ride_code_support,rental_uris,cross_street
0,1,"GRAN VIA CORTS CATALANES, 760",ELECTRICBIKESTATION,41.397978,2.180107,16.0,"GRAN VIA CORTS CATALANES, 760",8013,45,True,1000.0,True,NaN,NaN
1,2,"C/ ROGER DE FLOR, 126",ELECTRICBIKESTATION,41.395488,2.177198,17.0,"C/ ROGER DE FLOR, 126",8013,29,True,1000.0,True,NaN,NaN
2,3,"C/ NÀPOLS, 82",ELECTRICBIKESTATION,41.394156,2.181331,11.0,"C/ NÀPOLS, 82",8013,27,True,1000.0,True,NaN,NaN
3,4,"C/ RIBES, 13",ELECTRICBIKESTATION,41.393317,2.181248,8.0,"C/ RIBES, 13",8013,21,True,1000.0,True,NaN,NaN
4,5,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",ELECTRICBIKESTATION,41.391103,2.180176,7.0,"PG. LLUIS COMPANYS, 11 (ARC TRIOMF)",8018,39,True,1000.0,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,515,"C/ SANT ADRIÀ, 43",ELECTRICBIKESTATION,41.435207,2.194800,19.0,"C/ SANT ADRIÀ, 43",8030,24,True,1000.0,True,NaN,NaN
502,516,"C/ SANT ADRIÀ, 88",ELECTRICBIKESTATION,41.435460,2.200157,15.0,"C/ SANT ADRIÀ, 88",8030,21,True,1000.0,True,NaN,NaN
503,517,"AV. RASOS DE PEGUERA, 10",ELECTRICBIKESTATION,41.462095,2.178959,44.0,"AV. RASOS DE PEGUERA, 10",8033,20,True,1000.0,True,NaN,NaN
504,518,"C/ LLOBREGÓS, 115",ELECTRICBIKESTATION,41.424689,2.157049,112.0,"C/ LLOBREGÓS, 115",8032,27,True,1000.0,True,NaN,NaN


In [14]:
output_data = output_data.sort_values(by=['station_id'])
output_data


,index,station_id,last_reported,year,month,day,hour,percentage_docks_available
0,0,1,2022-12-31 22:59:21,2022,12,31,22,0.173913
41760763,1439769,1,2023-11-10 18:48:45,2023,11,10,18,0.800000
6100813,2036729,1,2023-02-14 23:56:13,2023,2,14,23,0.434783
38393589,2560201,1,2023-10-18 18:57:19,2023,10,18,18,0.818182
3279195,3279195,1,2023-01-23 18:52:28,2023,1,23,18,0.888889
...,...,...,...,...,...,...,...,...
5985672,1921588,520,2023-02-14 04:52:17,2023,2,14,4,0.071429
38568914,2735526,520,2023-10-19 13:42:57,2023,10,19,23,0.562500
5985167,1921083,520,2023-02-14 04:47:58,2023,2,14,4,0.071429
5984662,1920578,520,2023-02-14 04:43:41,2023,2,14,4,0.071429


In [15]:
output_data_mean = output_data.groupby(['station_id', 'year','month','day','hour']).mean().reset_index()[['station_id','year','month','day','hour','percentage_docks_available']]
output_data_mean

/var/folders/z9/sss87zk9299_8096st9s8gnh0000gn/T/ipykernel_7858/2268027269.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  output_data_mean = output_data.groupby(['station_id', 'year','month','day','hour']).mean().reset_index()[['station_id','year','month','day','hour','percentage_docks_available']]


,station_id,year,month,day,hour,percentage_docks_available
0,1,2022,12,31,22,0.173913
1,1,2022,12,31,23,0.183293
2,1,2023,1,1,0,0.144928
3,1,2023,1,1,1,0.157931
4,1,2023,1,1,2,0.136364
...,...,...,...,...,...,...
4116124,520,2023,10,26,12,0.469444
4116125,520,2023,10,26,13,0.400000
4116126,520,2023,10,26,14,0.333333
4116127,520,2023,11,28,11,0.602564


In [16]:
data['ctx-1'] = data['percentage_docks_available'].shift(1)
data['ctx-2'] = data['percentage_docks_available'].shift(2)
data['ctx-3'] = data['percentage_docks_available'].shift(3)
data['ctx-4'] = data['percentage_docks_available'].shift(4)


In [17]:
import dask.dataframe as dd 

df_info_stations = df_info_stations[['station_id', 'lat', 'lon']]
output_data = dd.merge(output_data, df_info_stations, on='station_id')


: 